In [ ]:
# ✅ 适配 OpenAI API，用于大文本分块处理
import openai
import json
import pandas as pd
import os

# ✅ 1. 设置 OpenAI API Key（推荐使用环境变量，避免泄露）
os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ✅ 2. 读取医学文本（逐块读取，防止超时）
file_path = "/content/Plumbs Veterinary Drug Handbook (Donald C. Plumb) (Z-Library) .txt"
chunk_size = 10000  # 每次处理 3000 字符
all_medicines = []  # 存储所有药物数据

with open(file_path, "r", encoding="utf-8") as f:
    while True:
        chunk = f.read(chunk_size)
        if not chunk:
            break  # 读取完成，退出循环

        # ✅ 3. 构建 Prompt（确保 GPT 严格输出 JSON）
        prompt = f"""
        你是一位专业的宠物医学和药物信息解析助手。请从以下医学文本中提取 **结构化药物信息**，包括：
        - **药物名称 (Medicine Name)**
        - **适应症 (UseCase)**
        - **禁忌 (Contraindications)**
        - **副作用 (SideEffects)**
        - **药理作用 (Pharmacology)**

        请严格按照 **JSON 格式** 返回，示例如下：
        ```json
        [
          {{
            "药物名称": "ACARBOSE",
            "适应症": ["用于糖尿病控制"],
            "禁忌": ["孕妇禁用", "对阿卡波糖过敏者禁用"],
            "副作用": ["胃肠道不适", "腹胀"],
            "药理作用": ["延缓碳水化合物的消化吸收"]
          }},
          ...
        ]
        ```
        **请直接返回 JSON，不要有额外的说明**。

        以下是医学文本：
        ```text
        {chunk}
        ```
        """

        # ✅ 4. 发送请求到 OpenAI GPT
        try:
            response = client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[
                    {"role": "system", "content": "你是专业的医学药物信息提取助手。"},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3  # 降低随机性，提高输出稳定性
            )

            # ✅ 5. 解析 GPT 结果（JSON 解析）
            gpt_response = response.choices[0].message.content.strip()
            parsed_data = json.loads(gpt_response)

            # ✅ 6. 合并当前块的数据
            all_medicines.extend(parsed_data)
            print(f"✅ 处理完成 {len(parsed_data)} 条药物数据，目前总数: {len(all_medicines)}")

        except json.JSONDecodeError:
            print("⚠ 解析 JSON 失败，可能 GPT 输出格式有误，跳过此部分！")
            print("🔥 GPT 输出内容：")
            print(gpt_response)  # 打印 GPT 输出，方便调试
            continue  # 继续处理下一块数据

# ✅ 7. 转换为 DataFrame 并展示部分数据
df_gpt = pd.DataFrame(all_medicines[:5])  # 仅展示前 5 条数据

# ✅ 8. 保存 JSON 文件
output_json = "parsed_medicine_data.json"
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(all_medicines, f, ensure_ascii=False, indent=4)

print(f"✅ 所有药物信息已提取完成，数据已保存到 {output_json}")


In [ ]:
# ✅ 适配 Google Colab，分块读取大文件
import openai
import json
import pandas as pd
import os
import asyncio
import nest_asyncio

# ✅ 让 Colab 兼容 asyncio
nest_asyncio.apply()

# ✅ 1. 设置 OpenAI API Key
os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ✅ 2. 读取医学文本（分块读取，防止内存溢出）
file_path = "Plumbs Veterinary Drug Handbook (Donald C. Plumb) (Z-Library).txt"
chunk_size = 10000  # 每次读取 10000 字符

def process_chunk(chunk_text):
    """同步调用 GPT 处理文本块（修复 await 兼容性）"""
    prompt = f"""
    你是一位专业的宠物医学和药物信息解析助手。请从以下医学文本中提取 **结构化药物信息**，包括：
    - **药物名称 (Medicine Name)**
    - **适应症 (UseCase)**
    - **禁忌 (Contraindications)**
    - **副作用 (SideEffects)**
    - **药理作用 (Pharmacology)**

    请严格按照 **JSON 格式** 返回，例如：
    [
      {{
        "药物名称": "ACARBOSE",
        "适应症": ["用于糖尿病控制"],
        "禁忌": ["孕妇禁用", "对阿卡波糖过敏者禁用"],
        "副作用": ["胃肠道不适", "腹胀"],
        "药理作用": ["延缓碳水化合物的消化吸收"]
      }},
      {{
        "药物名称": "...",
        "适应症": [...],
        "禁忌": [...],
        "副作用": [...],
        "药理作用": [...]
      }}
    ]

    **请直接返回 JSON，不要有额外的说明**。

    以下是医学文本：
    {chunk_text}
    """

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{"role": "system", "content": "你是专业的医学药物信息提取助手。"},
                  {"role": "user", "content": prompt}],
        temperature=0.3
    )

    try:
        gpt_response = response.choices[0].message.content
        parsed_data = json.loads(gpt_response)
        return parsed_data
    except Exception as e:
        print("⚠️ GPT 解析失败:", e)
        return []

def main():
    """同步读取大文本文件并解析"""
    all_medicines = []

    with open(file_path, "r", encoding="utf-8") as f:
        while True:
            chunk = f.read(chunk_size)  # 读取部分内容
            if not chunk:
                break  # 读完就退出循环

            medicines = process_chunk(chunk)  # 发送到 GPT 处理
            all_medicines.extend(medicines)  # 汇总所有解析数据

            print(f"✅ 处理完成 {len(all_medicines)} 种药物")

    # ✅ 4. 转换为 DataFrame
    df_gpt = pd.DataFrame(all_medicines)

    # ✅ 5. 保存 JSON 文件
    output_json = "parsed_medicine_data.json"
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(all_medicines, f, ensure_ascii=False, indent=4)

    print(f"✅ 所有药物信息已提取完成，共提取 {len(all_medicines)} 条数据，数据已保存到 {output_json}")

# ✅ 运行主任务
main()


In [ ]:
print(response_text)